# nanoServices: Komplettes Policy-Handling

Dieses Notebook zeigt, wie Policy-Management, Access Control, Contracts und Named-Graph-Mapping im nanoServices-Framework semantisch und interoperabel gelöst werden.

Wir setzen bevorzugt auf den ACP-Standard (Solid), erweitern aber für Verträge/Contracts mit ODRL und dokumentieren Ownership, Delegation, Named Graphs und Cross-System-Ressourcen.

Enthalten:
- Einführung & Grundprinzipien (ACP/ODRL)
- Turtle-Beispiele (ACP, ODRL, Named Graph Mapping)
- rdflib-Auswertung von Policies, Contracts, Ownership
- Erweiterungsideen (Delegation, Federation)

---

## 1. ACP- und ODRL-Prinzipien im nanoServices-Framework

**ACP** (Access Control Policies) ist der Standard für Zugriffsrechte und Policy-Handling im Solid-Ökosystem.

- Policies werden an Ressourcen (z. B. Named Graphs, Services) gebunden
- Rechte, Bedingungen und Agenten werden semantisch explizit hinterlegt

**ODRL** (Open Digital Rights Language) ist W3C-Standard für Verträge, Rechte und Pflichten.
- Perfekt für explizite, auditierbare Verträge/Agreements

---

## 2. Turtle: ACP-Policies, Contracts, Named Graph Mapping, Ownership

**Alle Beispiele sind kombinierbar und modular.**
- ACP = Standard-Policy-Layer
- ODRL = Vertrags-/Agreement-Layer
- nano: = eigene Erweiterung für Ownership, Delegation usw.

```turtle
@prefix acp: <http://www.w3.org/ns/solid/acp#> .
@prefix nano: <http://nanoservices.dev/ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix odrl: <http://www.w3.org/ns/odrl/2/> .

# System-Definition & Default-Graph-Mapping
nano:systemA a nano:System ;
    foaf:name "System A"@en ;
    nano:defaultGraph <http://fuseki.example.org/dataset/systemA/default> .

# ACP: Policy für den Default-Graph
nano:systemAPolicy a acp:Policy ;
    acp:allow acp:Read, acp:Write ;
    acp:anyOf ( nano:agentAlice nano:agentBob ) ;
    rdfs:label "Policy für System A Default-Graph"@de .

# ACP: AccessControl bindet Policy an Named Graph
nano:systemAAccessControl a acp:AccessControl ;
    acp:apply nano:systemAPolicy ;
    acp:access <http://fuseki.example.org/dataset/systemA/default> .

# ODRL: Vertrag für gemeinsamen Worker (Cross-System)
nano:sharedWorkerContract a odrl:Agreement ;
    odrl:permission [
        odrl:target nano:workerShared ;
        odrl:assignee nano:systemA, nano:systemB ;
        odrl:action odrl:use
    ] ;
    rdfs:label "Vertrag zur gemeinsamen Nutzung des Workers"@de .

# nano:Ownership, Delegation, System-Verknüpfung
nano:workerShared a nano:Worker ;
    rdfs:label "Shared Worker"@en ;
    nano:ownedBy nano:systemA, nano:systemB .

nano:delegationPolicy a acp:Policy ;
    acp:allow acp:Append ;
    acp:anyOf ( nano:partnerCompany ) ;
    nano:delegatedBy nano:systemA .

# Agenten & Firmen
nano:agentAlice a foaf:Person ; foaf:name "Alice" .
nano:agentBob a foaf:Person ; foaf:name "Bob" .
nano:partnerCompany a foaf:Organization ; foaf:name "Partner & Co KG" .
```


---
## 3. Python (rdflib): Auswertung von Policies, Contracts & Ownership

Dieses Beispiel liest das Turtle-Fragment ein und gibt die wichtigsten Policy-Zuordnungen aus.

Tipp: rdflib installieren, falls nicht vorhanden:
```python
!pip install rdflib
```


In [ ]:
# Turtle-Fragment in Variable einfügen (wie oben)
turtle_data = '''
@prefix acp: <http://www.w3.org/ns/solid/acp#> .
@prefix nano: <http://nanoservices.dev/ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix odrl: <http://www.w3.org/ns/odrl/2/> .

nano:systemA a nano:System ;
    foaf:name "System A"@en ;
    nano:defaultGraph <http://fuseki.example.org/dataset/systemA/default> .

nano:systemAPolicy a acp:Policy ;
    acp:allow acp:Read, acp:Write ;
    acp:anyOf ( nano:agentAlice nano:agentBob ) ;
    rdfs:label "Policy für System A Default-Graph"@de .

nano:systemAAccessControl a acp:AccessControl ;
    acp:apply nano:systemAPolicy ;
    acp:access <http://fuseki.example.org/dataset/systemA/default> .

nano:sharedWorkerContract a odrl:Agreement ;
    odrl:permission [
        odrl:target nano:workerShared ;
        odrl:assignee nano:systemA, nano:systemB ;
        odrl:action odrl:use
    ] ;
    rdfs:label "Vertrag zur gemeinsamen Nutzung des Workers"@de .

nano:workerShared a nano:Worker ;
    rdfs:label "Shared Worker"@en ;
    nano:ownedBy nano:systemA, nano:systemB .

nano:delegationPolicy a acp:Policy ;
    acp:allow acp:Append ;
    acp:anyOf ( nano:partnerCompany ) ;
    nano:delegatedBy nano:systemA .

nano:agentAlice a foaf:Person ; foaf:name "Alice" .
nano:agentBob a foaf:Person ; foaf:name "Bob" .
nano:partnerCompany a foaf:Organization ; foaf:name "Partner & Co KG" .
'''

from rdflib import Graph, Namespace

acp = Namespace("http://www.w3.org/ns/solid/acp#")
nano = Namespace("http://nanoservices.dev/ns#")
odrl = Namespace("http://www.w3.org/ns/odrl/2/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")

g = Graph()
g.parse(data=turtle_data, format="turtle")

print("ACP-AccessControls, Policies & zugehörige Named Graphs:\n")
for ac, _, policy in g.triples((None, acp.apply, None)):
    for _, _, graph_uri in g.triples((ac, acp.access, None)):
        print(f"AccessControl: {ac}\n  Policy: {policy}\n  Named Graph: {graph_uri}\n---")

print("\nODRL Agreements (Contracts) für Shared Worker:\n")
for contract in g.subjects(predicate=None, object=odrl.Agreement):
    print(f"Contract: {contract}")
    for perm in g.objects(subject=contract, predicate=odrl.permission):
        for target in g.objects(subject=perm, predicate=odrl.target):
            print(f"  Target: {target}")
        for assignee in g.objects(subject=perm, predicate=odrl.assignee):
            print(f"  Assignee: {assignee}")
        for action in g.objects(subject=perm, predicate=odrl.action):
            print(f"  Action: {action}")
    print("---")

print("\nOwnership/Delegation Übersicht:\n")
for worker in g.subjects(predicate=RDF.type, object=nano.Worker):
    owners = list(g.objects(subject=worker, predicate=nano.ownedBy))
    print(f"Worker {worker} owned by: {owners}")

for policy in g.subjects(predicate=RDF.type, object=acp.Policy):
    delegator = list(g.objects(subject=policy, predicate=nano.delegatedBy))
    if delegator:
        print(f"Policy {policy} delegated by: {delegator}")

---
## 4. Weiterführende Aufgaben & Ausblick

- Policies/Contracts dynamisch abfragen, visualisieren oder exportieren
- Mehrstufige Delegationen/Trust-Chains zwischen Systemen modellieren
- Integration mit ACP-Implementierungen (z. B. Solid-Pods, Community-Server)
- Eigene nano:-Extensions für Spezialfälle dokumentieren und (wo möglich) mit ACP/ODRL verknüpfen

---

**Fragen, Anpassungswünsche oder Deep-Dive in einzelne Bereiche? Einfach im Notebook ergänzen oder im Chat anfragen!**